<a href="https://colab.research.google.com/github/adamrhurley/FYP/blob/main/MusicRecommenderSystem/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#References:
# https://machinelearningmastery.com/tutorial-to-implement-k-nearest-neighbors-in-python-from-scratch/
# https://realpython.com/knn-python/

In [ ]:
import pandas as pd
import numpy as np
import io

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving artists.csv to artists.csv
Saving userArtists.csv to userArtists.csv


In [ ]:
userArtists = pd.read_csv(io.BytesIO(uploaded['userArtists.csv']))
artists = pd.read_csv(io.BytesIO(uploaded['artists.csv']))

In [ ]:
data = pd.merge(userArtists, artists, on='artistID')

In [ ]:
userArtists.head()

,userID,artistID,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983


In [ ]:
artists.head()

,artistID,name,url,pictureURL
0,1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg
1,2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg
2,3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...
3,4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...
4,5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...


In [ ]:
data.head()

,userID,artistID,weight,name,url,pictureURL
0,2,51,13883,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg
1,4,51,228,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg
2,27,51,85,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg
3,28,51,10,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg
4,62,51,528,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg


In [ ]:
artists['total_plays'] = userArtists.weight.value_counts()
artists['mean_plays'] = userArtists.groupby('artistID').mean()['weight']
artists.head()

,artistID,name,url,pictureURL,total_plays,mean_plays
0,1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg,NaN,NaN
1,2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg,636.0,257.000000
2,3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...,419.0,667.666667
3,4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...,338.0,258.333333
4,5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...,345.0,281.500000


In [ ]:
def getDistance(x, y):
    return np.linalg.norm(x - y)

def userPlays(user_id):
    selectedUser = userArtists[ userArtists.userID == user_id ]
    return selectedUser.drop(['userID'], axis=1)


def mergeDatasets(df1,df2):
    return df1.merge(df2, how='inner', on="artistID")

def usersDisatnces(user_id1, user_id2):
    vector_user1 = userPlays(user_id1)
    vector_user2 = userPlays(user_id2)
    vectors = mergeDatasets(vector_user1, vector_user2)
    
    if vectors.empty:
        return (user_id1, user_id2, 100)
    return user_id1, user_id2, getDistance(vectors['weight_x'], vectors['weight_y'])

def getID():
    return list(userArtists.userID.unique())

def allDistances(user_id):
    if not user_id in userArtists.userID:
        return False   
    
    ids = getID()
    return [usersDisatnces(user_id, for_id) for for_id in ids[:10]]

def artistsRecommendations(userID, k=5):
    allDistUser = allDistances(userID)
    allDistUser = pd.DataFrame(allDistUser, columns=['ID1','userID','distance'])
    allDistUser.sort_values('distance', axis=0, ascending=True, inplace=True)
    artist_list = list()

    plays = userPlays(userID)
    allUsers = list(allDistUser.userID)

    for user in allUsers:
        merged = plays.merge(userPlays(user), how='right', on="artistID")
        merged = merged[merged.weight_x.isna()]
        artist_list.append(merged)
        artist_list = pd.concat(artist_list)
        return artists.loc[artist_list.artistID].sort_values('mean_plays', ascending=False)[:k]

In [ ]:
artistsRecommendations(43)

,artistID,name,url,pictureURL,total_plays,mean_plays
101,107,Burial,http://www.last.fm/music/Burial,http://userserve-ak.last.fm/serve/252/60689649...,176.0,2981.400000
107,113,Dustin O'Halloran,http://www.last.fm/music/Dustin+O%27Halloran,http://userserve-ak.last.fm/serve/252/28429321...,190.0,849.666667
102,108,Magnitophono,http://www.last.fm/music/Magnitophono,http://userserve-ak.last.fm/serve/252/24388525...,190.0,662.000000
103,109,andy graydon,http://www.last.fm/music/andy+graydon,http://userserve-ak.last.fm/serve/252/26569717...,205.0,541.250000
104,110,ddekombinacja,http://www.last.fm/music/ddekombinacja,http://userserve-ak.last.fm/serve/252/31731637...,174.0,431.000000
